In [1]:
import pandas as pd
from keras.layers import Input, Embedding, LSTM, Dense, Dot, Softmax, Concatenate, BatchNormalization, Attention
from keras.models import Model
import tensorflow as tf
import numpy as np
from keras.utils import pad_sequences

In [2]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'orders_and_products_by_customer.pkl')

In [3]:
product_data = product_data.sample(n=1000)
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
161006,7014717.0,[7014683],"[[6556091, 6143081]]","[[Elegante Stiefelette Rauleder Blau, Elegante...","[[1, 1]]","[[125.0, 130.0]]","[[1633338269, 1633338269]]"
110174,5808187.0,[5808183],[[407457]],[[Pantolette Rauleder grau]],[[1]],[[69.95]],[[1620226968]]
117632,5977435.0,[5977432],[[4772019]],[[Riemchensandale Rauleder rot]],[[1]],[[99.95]],[[1622179990]]
211554,8237133.0,[8237129],"[[7699357, 7793733]]","[[Sneaker low Materialmix Leder Blau, Sneaker ...","[[1, 1]]","[[99.95, 140.0]]","[[1648661921, 1648661921]]"
273674,9570198.0,[9570195],[[8807325]],[[Sneaker low Rauleder grün]],[[1]],[[99.95]],[[1663513869]]
...,...,...,...,...,...,...,...
135463,6446020.0,[6446015],"[[5149676, 5375078, 4998087]]","[[Pantolette Glattleder braun, Pantolette Raul...","[[1, 1, 1]]","[[59.0, 59.0, 69.0]]","[[1626721332, 1626721332, 1626721332]]"
319010,10305780.0,[10305777],[[8504072]],[[Winterstiefelette Rauleder beige]],[[1]],[[125.0]],[[1670843460]]
201844,8002533.0,[8002530],[[556949]],[[Eleganter Pumps Glattleder schwarz]],[[1]],[[89.95]],[[1645792116]]
3248,1978282.0,"[5518366, 7540078, 7886109]","[[4775784, 5338399], [3548784, 4030395, 494660...","[[Spangenpumps Rauleder rosa, Riemchensandale ...","[[1, 1], [1, 1, 1], [1, 1]]","[[99.95, 99.95], [99.0, 69.0, 79.0], [89.0, 69...","[[1617357897, 1617357897], [1637917892, 163791..."


In [4]:
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
161006,7014717.0,[7014683],"[[6556091, 6143081]]","[[Elegante Stiefelette Rauleder Blau, Elegante...","[[1, 1]]","[[125.0, 130.0]]","[[1633338269, 1633338269]]"
110174,5808187.0,[5808183],[[407457]],[[Pantolette Rauleder grau]],[[1]],[[69.95]],[[1620226968]]
117632,5977435.0,[5977432],[[4772019]],[[Riemchensandale Rauleder rot]],[[1]],[[99.95]],[[1622179990]]
211554,8237133.0,[8237129],"[[7699357, 7793733]]","[[Sneaker low Materialmix Leder Blau, Sneaker ...","[[1, 1]]","[[99.95, 140.0]]","[[1648661921, 1648661921]]"
273674,9570198.0,[9570195],[[8807325]],[[Sneaker low Rauleder grün]],[[1]],[[99.95]],[[1663513869]]
...,...,...,...,...,...,...,...
135463,6446020.0,[6446015],"[[5149676, 5375078, 4998087]]","[[Pantolette Glattleder braun, Pantolette Raul...","[[1, 1, 1]]","[[59.0, 59.0, 69.0]]","[[1626721332, 1626721332, 1626721332]]"
319010,10305780.0,[10305777],[[8504072]],[[Winterstiefelette Rauleder beige]],[[1]],[[125.0]],[[1670843460]]
201844,8002533.0,[8002530],[[556949]],[[Eleganter Pumps Glattleder schwarz]],[[1]],[[89.95]],[[1645792116]]
3248,1978282.0,"[5518366, 7540078, 7886109]","[[4775784, 5338399], [3548784, 4030395, 494660...","[[Spangenpumps Rauleder rosa, Riemchensandale ...","[[1, 1], [1, 1, 1], [1, 1]]","[[99.95, 99.95], [99.0, 69.0, 79.0], [89.0, 69...","[[1617357897, 1617357897], [1637917892, 163791..."


In [5]:
display(product_data[['customer_id', 'order_ids', 'product_ids', 'amounts', 'order_dates']])

,customer_id,order_ids,product_ids,amounts,order_dates
161006,7014717.0,[7014683],"[[6556091, 6143081]]","[[1, 1]]","[[1633338269, 1633338269]]"
110174,5808187.0,[5808183],[[407457]],[[1]],[[1620226968]]
117632,5977435.0,[5977432],[[4772019]],[[1]],[[1622179990]]
211554,8237133.0,[8237129],"[[7699357, 7793733]]","[[1, 1]]","[[1648661921, 1648661921]]"
273674,9570198.0,[9570195],[[8807325]],[[1]],[[1663513869]]
...,...,...,...,...,...
135463,6446020.0,[6446015],"[[5149676, 5375078, 4998087]]","[[1, 1, 1]]","[[1626721332, 1626721332, 1626721332]]"
319010,10305780.0,[10305777],[[8504072]],[[1]],[[1670843460]]
201844,8002533.0,[8002530],[[556949]],[[1]],[[1645792116]]
3248,1978282.0,"[5518366, 7540078, 7886109]","[[4775784, 5338399], [3548784, 4030395, 494660...","[[1, 1], [1, 1, 1], [1, 1]]","[[1617357897, 1617357897], [1637917892, 163791..."


In [6]:
product_data.order_ids

161006                      [7014683]
110174                      [5808183]
117632                      [5977432]
211554                      [8237129]
273674                      [9570195]
                     ...             
135463                      [6446015]
319010                     [10305777]
201844                      [8002530]
3248      [5518366, 7540078, 7886109]
36721                       [4089743]
Name: order_ids, Length: 1000, dtype: object

In [7]:
print(product_data.product_ids, product_data.order_dates)

161006                                 [[6556091, 6143081]]
110174                                           [[407457]]
117632                                          [[4772019]]
211554                                 [[7699357, 7793733]]
273674                                          [[8807325]]
                                ...                        
135463                        [[5149676, 5375078, 4998087]]
319010                                          [[8504072]]
201844                                           [[556949]]
3248      [[4775784, 5338399], [3548784, 4030395, 494660...
36721                         [[4064235, 4031882, 4078253]]
Name: product_ids, Length: 1000, dtype: object 161006                           [[1633338269, 1633338269]]
110174                                       [[1620226968]]
117632                                       [[1622179990]]
211554                           [[1648661921, 1648661921]]
273674                                       [[166351

In [8]:
def bringToSameSizeOneDim(column):
    max_length = max(column.apply(len))
    return column.apply(lambda x: x + [0] * (max_length - len(x)))


def bringToSameSizeTwoDimAndFlatten(column):
    max_length = max(column.apply(lambda x: max(len(sublist) for sublist in x)))
    new_column = column.apply(lambda x: [sublist + [0] * (max_length - len(sublist)) for sublist in x])
    new_column = new_column.apply(lambda x: x + [[0] * max_length] * (max_length - len(x)))
    return new_column.apply(lambda x: [item for sublist in x for item in sublist])

product_data.order_ids = bringToSameSizeOneDim(product_data.order_ids)
product_data.product_ids = bringToSameSizeTwoDimAndFlatten(product_data.product_ids)
product_data.amounts = bringToSameSizeTwoDimAndFlatten(product_data.amounts)
product_data.total_prices = bringToSameSizeTwoDimAndFlatten(product_data.total_prices)
product_data.order_dates = bringToSameSizeTwoDimAndFlatten(product_data.order_dates)




In [9]:
def convertToSequence(column):
    list = column.tolist()
    max_length = max(len(x) for x in list)
    return pad_sequences(list, maxlen=max_length, padding='post', value=0), max_length

products_inputs, max_length_products = convertToSequence(product_data.product_ids)
dates_inputs, max_length_dates = convertToSequence(product_data.order_dates)
target_data, _ = convertToSequence(product_data.amounts)




In [10]:
display(max_length_dates, dates_inputs.shape[1], target_data.shape)
display(target_data)
display(dates_inputs)
display(products_inputs)
# 1602099712

50

50

(1000, 50)

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)

array([[1633338269, 1633338269,          0, ...,          0,          0,
                 0],
       [1620226968,          0,          0, ...,          0,          0,
                 0],
       [1622179990,          0,          0, ...,          0,          0,
                 0],
       ...,
       [1645792116,          0,          0, ...,          0,          0,
                 0],
       [1617357897, 1617357897,          0, ...,          0,          0,
                 0],
       [1600358792, 1600358792, 1600358792, ...,          0,          0,
                 0]], dtype=int32)

array([[6556091, 6143081,       0, ...,       0,       0,       0],
       [ 407457,       0,       0, ...,       0,       0,       0],
       [4772019,       0,       0, ...,       0,       0,       0],
       ...,
       [ 556949,       0,       0, ...,       0,       0,       0],
       [4775784, 5338399,       0, ...,       0,       0,       0],
       [4064235, 4031882, 4078253, ...,       0,       0,       0]],
      dtype=int32)

In [11]:
# Define the model architecture
# num_orders = sum(len(order) for order in product_data.order_ids)
# num_products =  sum(len(order) for order in product_data.product_ids)
# num_units = len([amount for order_amounts in product_data.amounts for amount in order_amounts])
# num_prices = len([amount for order_amounts in product_data.total_prices for amount in order_amounts])
# num_dates = len([amount for order_amounts in product_data.order_dates for amount in order_amounts])
#
# print(num_orders, num_products, num_units, num_prices, num_dates)

In [17]:
num_dates = np.max(dates_inputs) + 1
dates_one_hot = np.eye(num_dates)[dates_inputs]

# Convert the product_ids array to a one-hot encoded sequence
num_products = np.max(products_inputs) + 1
product_ids_one_hot = np.eye(num_products)[products_inputs]

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [ ]:
input_dates = Input(shape=(dates_one_hot.shape[1], dates_one_hot.shape[2]))
input_product_ids = Input(shape=(product_ids_one_hot.shape[1], product_ids_one_hot.shape[2]))

# LSTM layer for processing dates
lstm_dates = LSTM(64, return_sequences=True)(input_dates)

# LSTM layer for processing product_ids
lstm_product_ids = LSTM(64, return_sequences=True)(input_product_ids)

# Attention layer
attention = Attention()([lstm_dates, lstm_product_ids])

# Output layer
output = Dense(num_products, activation='softmax')(attention)

# Create the model
model = Model(inputs=[input_dates, input_product_ids], outputs=output)
model.summary()

In [ ]:
model.fit([dates_inputs, products_inputs], target_data, epochs=10, batch_size=32)  # Adjust epochs and batch size as needed

In [ ]:
# Input layers
date_inputs = Input(shape=dates_inputs.shape)
# product_inputs = Input(shape=(num_orders, num_units))
product_inputs = Input(products_inputs.shape)
# price_inputs = Input(shape=(num_orders, num_prices))

encoding_padding_mask = tf.math.logical_not(tf.math.equal(product_inputs, 0))

# Embedding layers
date_emb = Embedding(num_dates, 16, input_length=num_dates)(date_inputs)
product_emb = Embedding(num_products, 16, input_length=num_units)(product_inputs)

concat_embedding_input = Concatenate(
    name='concat_embedding_input')([date_emb, product_emb])

concat_embedding_input = BatchNormalization(
    name='batchnorm_inputs')(concat_embedding_input)
# LSTM layer

lstm = LSTM(64, return_sequences=True)(concat_embedding_input)

lstm = BatchNormalization(name='batchnorm_lstm')(lstm)

att = Attention(use_scale=False,
                                name='attention')(inputs=[lstm, lstm],
                                                  mask=[encoding_padding_mask,
                                                        encoding_padding_mask])


# Concatenate layer
#concat = Concatenate(axis=2)([context, price_inputs])

# Dense layer
output = Dense(1)(att)
# Define the model
model = Model(inputs=[date_inputs, product_inputs], outputs=output)
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

In [ ]:
# Fit the model
model.fit(x=[dates_inputs, products_inputs], y=target_data, epochs=10, batch_size=32)  # Adjust epochs and batch size as needed


In [ ]:
#display(X_train)
#output

# product_ids_flat = product_data.product_ids.to_numpy().reshape(-1)
# order_dates_flat = product_data.order_dates.to_numpy().reshape(-1)

# Create the output variable
output_variable = product_data.amounts.to_numpy().reshape(-1, 1)

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((product_data.order_dates.to_numpy(), product_data.product_ids.to_numpy(), output_variable))

# Fit the model
history = model.fit(dataset, epochs=50, batch_size=32)


In [ ]:
display(order_dates_flat)

In [ ]:
y = []
output_variable = product_data.amounts.to_numpy().reshape(-1, 1)
dataset = tf.data.Dataset.from_tensor_slices((np.array(product_data.order_dates), np.array(product_data.product_ids), output_variable))
history = model.fit(dataset, epochs=50, batch_size=32)

In [ ]:
# Evaluate the model
loss, mse = model.evaluate([product_data.order_dates_test, product_data.product_ids_test, product_data.total_prices_test], product_data.total_prices_test)
print("Test loss: {:.4f}, Test MSE: {:.4f}".format(loss, mse))

In [ ]:
# split the data into training and testing sets
#X = pd.get_dummies(product_data['product__id'], prefix='product')
#y = product_data['amount']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)